In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# Bigquery
## Load the Data for 8 schools' data from Bigquery

In [ ]:
'''
%%bigquery --project ds-student-data df
SELECT *
FROM `ds-student-data.Distinctive.ChronicAbsence` 
'''

'\n%%bigquery --project ds-student-data df\nSELECT *\nFROM `ds-student-data.Distinctive.ChronicAbsence` \n'

In [ ]:
#Getting the ChronicAbsence data
#Change the last where clause by condition based on the past days of the current acadmic year

%%bigquery --project ds-student-data df

SELECT a.NameOfInstitution,a.StudentUniqueId, sde.GradeLevel, a.LastSurname, a.FirstName, a.NumberOfTotalAbsences, b.AverageAbsence
FROM `ds-student-data.StarterPack.StudentAttendance` as a
INNER JOIN
(SELECT ROUND(AVG(NumberOfTotalAbsences),1) AverageAbsence, NameOfInstitution
FROM `ds-student-data.StarterPack.StudentAttendance`
GROUP BY NameOfInstitution ) as b
ON a.NameOfInstitution = b.NameOfInstitution
LEFT JOIN `ds-student-data.StarterPack.StudentDemographicsAndEnrollment` as sde
ON a.StudentUniqueID=sde.StudentUniqueID
WHERE a.NumberOfTotalAbsences >=6

In [ ]:
df.shape

(1380, 7)

In [ ]:
df.groupby('NameOfInstitution')['StudentUniqueId'].count()

NameOfInstitution
Bucktown                                 147
Distinctive College Prep Harper Woods    316
Distinctive College Prep Redford         136
Irving Park                              107
Plymouth Educational Center              378
Prairie                                  150
River Heights Academy                    104
West Belden                               42
Name: StudentUniqueId, dtype: int64

In [ ]:
df['NameOfInstitution'].unique()

array(['Distinctive College Prep Harper Woods',
       'Distinctive College Prep Redford', 'Prairie', 'Bucktown',
       'Irving Park', 'West Belden', 'Plymouth Educational Center',
       'River Heights Academy'], dtype=object)

In [ ]:
#df.shape

In [ ]:
!ls /content/gdrive/My\ Drive/Chronic_absenteeism_letter/

 2020-01-23		  3.2ChronicAbsence.csv
 2020-02-07		 'AIM Attendance 12.10.csv'
 2020-02-13		 'CA 12-14-20.gdoc'
 2020-02-25		 'Chronic Absence - Absence List Import.csv'
 2020-03-03		 'Chronic Absence Letter #1 - Jan '\''20.gdoc'
 2020-12-16		 'Chronic Absence.xlsx'
 2.13ChronicAbsence.csv   Chronic_Absenteeism.ipynb


In [ ]:
schoolname = {'Prairie':'CICS Prairie', 'Bucktown':'CICS Bucktown', 'Distinctive College Prep Harper Woods':'DCP Harper Woods',
       'Distinctive College Prep Redford':'DCP Redford', 'West Belden':'CICS West Belden', 'Irving Park':'CICS Irving Park', 'Plymouth Educational Center':'Plymouth Educational Center','River Heights Academy':'River Heights Academy'}

In [ ]:
df['NameOfInstitution'] = df['NameOfInstitution'].map(schoolname)

# Load from CSV

##use this code to load data is not on bigquery: AIM

In [ ]:
#Data saved in csv/excel
import pandas as pd
import glob, os

data_dir= '/content/gdrive/My Drive/Chronic_absenteeism_letter/'
filen = 'AIM Attendance 12.10.csv'

os.chdir(data_dir)

adf=pd.read_csv(filen)

print("Column headings:")
print(adf.columns)


Column headings:
Index(['NameOfInstitution', 'Perm ID', 'Grade', 'LastSurname', 'FirstName',
       'NumberOfTotalAbsences', 'Unnamed: 6'],
      dtype='object')


In [ ]:
adf.shape

(257, 7)

In [ ]:
df.columns

Index(['NameOfInstitution', 'StudentUniqueId', 'GradeLevel', 'LastSurname',
       'FirstName', 'NumberOfTotalAbsences', 'AverageAbsence'],
      dtype='object')

In [ ]:
adf.columns =['NameOfInstitution', 'StudentUniqueId', 'GradeLevel', 'LastSurname',
       'FirstName', 'NumberOfTotalAbsences', 'AverageAbsence']

In [ ]:
adf.groupby('NameOfInstitution')['NumberOfTotalAbsences'].mean().round(1)

NameOfInstitution
Art in Motion    21.3
Name: NumberOfTotalAbsences, dtype: float64

In [ ]:
#Combine the 9 schools 
frames = [df, adf]
df = pd.concat(frames)

In [ ]:
df.tail()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
252,Art in Motion,50199175,9,Williams,Jahzlynn,5.0,21.0
253,Art in Motion,50123596,9,Williams,Tamiyah,46.0,21.0
254,Art in Motion,50301647,9,Willis,Malaysia,44.0,21.0
255,Art in Motion,50263275,9,Woods,Collin,25.0,21.0
256,Art in Motion,45428796,9,Woods,Jordyn,40.0,21.0


#Getting the Images  & Principal info

In [ ]:
import pandas as pd
import glob, os

data_dir= '/content/gdrive/My Drive/Chronic_absenteeism_letter/'
filename = 'Chronic Absence.xlsx'

os.chdir(data_dir)


In [ ]:
dname=pd.read_excel(filename, sheet_name='Images  & PrincipalTable')

In [ ]:
df.NameOfInstitution.unique()

array(['DCP Harper Woods', 'DCP Redford', 'CICS Prairie', 'CICS Bucktown',
       'CICS Irving Park', 'CICS West Belden',
       'Plymouth Educational Center', 'River Heights Academy',
       'Art in Motion'], dtype=object)

In [ ]:
dc=df.groupby("NameOfInstitution")['AverageAbsence'].mean().round(1).to_dict()

In [ ]:
dc

{'Art in Motion': 21.0,
 'CICS Bucktown': 4.0,
 'CICS Irving Park': 3.3,
 'CICS Prairie': 6.4,
 'CICS West Belden': 1.7,
 'DCP Harper Woods': 12.5,
 'DCP Redford': 12.9,
 'Plymouth Educational Center': 17.9,
 'River Heights Academy': 9.5}

In [ ]:
#Getting the total number of absence for all schools
%%bigquery --project ds-student-data mni

SELECT AVG(NumberOfTotalAbsences) as avgDS
FROM `ds-student-data.StarterPack.StudentAttendance`

#If you need to add a fixed number write over mni.avgDS.loc[0]....

In [ ]:
wholeavg = mni.avgDS.loc[0].round(1)
wholeavg

7.7

In [ ]:
for school in dc:
  if dc[school] > wholeavg:
    dc[school] = wholeavg

In [ ]:
dc

{'Art in Motion': 7.7,
 'CICS Bucktown': 4.0,
 'CICS Irving Park': 3.3,
 'CICS Prairie': 6.4,
 'CICS West Belden': 1.7,
 'DCP Harper Woods': 7.7,
 'DCP Redford': 7.7,
 'Plymouth Educational Center': 7.7,
 'River Heights Academy': 7.7}

In [ ]:
df['AverageAbsence'] = df['NameOfInstitution'].map(dc)

In [ ]:
df.head()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
0,DCP Harper Woods,50000994,K,Summers,Kristian,14.0,7.7
1,DCP Harper Woods,60000006,K,Chambers,Royal,28.0,7.7
2,DCP Harper Woods,50000689,6,Webster,Khamiah,18.0,7.7
3,DCP Harper Woods,50000689,6,Webster,Khamiah,18.0,7.7
4,DCP Harper Woods,50000992,K,Brown,Skylar,11.0,7.7


In [ ]:
links = dict(zip(dname.Campus, dname.Link))

In [ ]:
links

{'Art in Motion': 'https://www.distinctiveschools.org/uploads/8/4/9/1/84911918/published/aim-circle-logo-color-withwhite.png?1581611169',
 'CICS Bucktown': 'https://www.cicsbucktown.org/uploads/8/4/9/1/84911918/published/cics-bucktown-horz.png?1531843975',
 'CICS Irving Park': 'https://www.cicsirvingpark.org/uploads/8/4/9/1/84911918/published/cics-irving-park-horz.png?1531844890',
 'CICS Prairie': 'https://www.cicsprairie.org/uploads/8/4/9/1/84911918/published/cics-prairie-horz.png?1531844962',
 'CICS West Belden': 'https://www.cicswestbelden.org/uploads/8/4/9/1/84911918/published/cics-west-belden-horz.png?1531845057',
 'DCP Harper Woods': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png',
 'DCP Redford': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png',
 'Plymouth Educational Center': 'https://www.plymouthed.org/uploads/8/4/9/1/84911918/published/pec-navylogo_1.png?1561052576',
 'River Heights Academy': 'https://

In [ ]:
#Make a dictionary matching the campus to Director names
dnames = dict(zip(dname.Campus, dname.Director))

In [ ]:
dnames

{'Art in Motion': 'Ms. Kara May',
 'CICS Bucktown': "Ms. Sarah O'Connell",
 'CICS Irving Park': 'Mr. JW Kuebler',
 'CICS Prairie': 'Ms. Jennifer Harth',
 'CICS West Belden': 'Ms. Colleen Collins',
 'DCP Harper Woods': 'Ms. Cassie Williams',
 'DCP Redford': 'Ms. Camille Hibbler',
 'Plymouth Educational Center': 'Ms. Schranda Battle',
 'River Heights Academy': 'Ms. Jessica Kull'}

In [ ]:
mails = {'CICS Bucktown':'soconnell@distinctiveschools.org','CICS Irving Park': 'jkuebler@distinctiveschools.org',
 'CICS Prairie': 'jharth@distinctiveschools.org',
 'CICS West Belden': 'ccollins@distinctiveschools.org',
 'DCP Harper Woods': 'cwilliams@distinctiveschools.org',
 'DCP Redford': 'chibbler@distinctiveschools.org' ,
 'Art in Motion': 'kmay@distinctiveschools.org',
 'Plymouth Educational Center': 'sbattle@distinctiveschools.org' ,
 'River Heights Academy': 'jkull@distinctiveschools.org'}

# Create the letter

In [ ]:
!pip3 install reportlab

     |████████████████████████████████| 2.6MB 9.6MB/s 


In [ ]:
import os

def createFolder(directory):

    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        



In [ ]:
maxnumber = df['NumberOfTotalAbsences'].max()
maxnumber

65.0

In [ ]:
from reportlab.lib import colors
from reportlab.graphics.shapes import Drawing, String
from reportlab.graphics.charts.barcharts import HorizontalBarChart


def make_drawing(row):
    drawing = Drawing()
    data = [(row['NumberOfTotalAbsences'], row['AverageAbsence']),]
    names = [row['FirstName'], 'Classmates\n /Compañeros']
    
    bc = HorizontalBarChart()
    bc.x = 20
    bc.y = 50
    bc.height = 150
    bc.width = 400
    bc.data = data
    bc.strokeColor = colors.black
    bc.valueAxis.valueMin = 0
    bc.valueAxis.visibleGrid = True
    bc.valueAxis.valueMax = maxnumber
    bc.valueAxis.valueStep = 10

    bc.barLabels.fontName = "Helvetica"
    bc.barLabels.fontSize = 12
    bc.barLabelFormat = '%d'
    bc.barLabels.nudge = 7
    
    bc.categoryAxis.labels.boxAnchor = 'ne'
    bc.categoryAxis.labels.dx = -10
    bc.categoryAxis.labels.dy = 0 
    bc.categoryAxis.labels.fontName = 'Helvetica'
    bc.categoryAxis.categoryNames = names
    
    bc.valueAxis.visible = True
    #Customize the color of the bars  
    bc.bars[(0, 0)].fillColor = colors.steelblue 
    bc.bars[(0, 1)].fillColor = colors.lightsteelblue
    drawing.add(bc)

    return drawing

In [ ]:
import datetime
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import PageBreak
from reportlab.lib.utils import ImageReader
 
def build_a_letter(row, test=True):
    '''
    Input: dataframe row
    Output: pdf file named after studentID
    '''
    #I name the pdf name after the Initals and StudentID
  
    name = str(row['FirstName'][0])+ str(row['LastSurname'][0])+str(row['StudentUniqueId'])+'.pdf'
    full_name = row['FirstName']+' ' + row['LastSurname']
    school_name = row.NameOfInstitution
    grade = str(row.GradeLevel)
    formatted_time = datetime.date.today().isoformat()
    
    if test:
      outfiledir = data_dir+ 'test'+'/'+str(school_name)+'/'+grade+'/'
    else:
      outfiledir = data_dir+ str(formatted_time)+'/'+str(school_name)+'/'+grade+'/'
    createFolder(outfiledir)
    outfilepath = os.path.join(outfiledir, name)
    #print(outfilepath)

    doc = SimpleDocTemplate(outfilepath, pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    Story=[]

    if school_name == 'DCP Harper Woods' or school_name =='DCP Redford':
      logo = Image(links[school_name], width=2*inch, height=0.42*inch, kind='proportional')
    else:
      logo = Image(links[school_name])

    
    NumberOfTotalAbsences = str(row.NumberOfTotalAbsences)
    story = []
    #Set the logo 
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
     
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    ptext = '<font size=11>We are writing to you because <u>our records show that your child has missed more than 10 % of the school year to date.</u> During this challenging pandemic, we recognize the difficulties of transitioning to a very different school year in 2020. In light of the current COVID circumstances, class attendance and participation remains critical for a child’s educational growth and achievement, and we are committed to your child’s success.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Our team will be following up this letter with a call to discuss reasons behind your student’s absence from remote learning. We recognize that the COVID pandemic has introduced a host of new challenges that our students and families have to navigat. Our goal is to work with you to identify the factors contributing to your child’s absenteeism and work together on a solution for improvement.  If absences continue, it will be necessary to schedule a meeting with school administration and commit to a further plan of action.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Thank you for your partnership in addressing this matter, and thank you in advance for your dedication to ensuring that your child is set up for success in school this year. We are #BetterTogether, and we look forward to partnering with you to ensure that your child has a great year of learning and growth. </font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=11>Sincerely,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 
    
    #Second page spanish version
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11><u><b>Primera notificación</b></u></font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Estimada familia de %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext ='<font size=11>Le escribimos porque <u>nuestros registros muestran que \
    su hijo ha perdido más del 10 % del año escolar hasta la fecha.</u> Por favor, \
    comprenda que todas las ausencias se consideran injustificadas hasta que se \
    presente la documentación adecuada de acuerdo con el manual de la escuela. \
    La asistencia regular a la escuela es fundamental para el crecimiento y el \
    rendimiento educativo de un niño, y estamos comprometidos con el éxito de su\
     hijo. Esperamos asociarnos con usted este año para asegurar que su hijo \
     continúe teniendo un gran año de aprendizaje y crecimiento.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Si las ausencias continúan, programaremos un horario \
    para que usted venga a la escuela a una reunión con nuestra administración \
    con el objetivo de identificar los factores que contribuyen al absentismo de\
     su hijo, discutir formas en que podemos mejorar la asistencia de su hijo \
     este año y comprometerse a un plan de acción juntos. Estos pasos se tomarán\
      de acuerdo con la política de asistencia de Distinctive Schools. </font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Gracias por su atención a este asunto y gracias de \
    antemano por su colaboración y dedicación para garantizar que su hijo esté \
    preparado para el éxito en la escuela este año. Espero con interés trabajar \
    con usted para asegurarnos de que su hijo tenga un año positivo y productivo.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=11>Atentamente,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 

    #Third page barchart
    title_style = styles['Heading1']
    title_style.alignment = 1
    title = Paragraph("Absences Summary/Resumen de ausencias", title_style)
    Story.append(title)
    Story.append(Spacer(1, 24))
    pic = make_drawing(row)
    Story.append(pic)

    doc.build(Story)

In [ ]:
def build_english_letter(row, test=True):
    '''
    Input: dataframe row
    Output: pdf file named after studentID
    '''
    #I name the pdf name after the Initals and StudentID
  
    name = str(row['FirstName'][0])+ str(row['LastSurname'][0])+str(row['StudentUniqueId'])+'.pdf'
    full_name = row['FirstName']+' ' + row['LastSurname']
    school_name = row.NameOfInstitution
    grade = str(row.GradeLevel)
    formatted_time = datetime.date.today().isoformat()
    
    if test:
      outfiledir = data_dir+ 'test'+'/'+str(school_name)+'/'+grade+'/'
    else:
      outfiledir = data_dir+ str(formatted_time)+'/'+str(school_name)+'/'+grade+'/'
    createFolder(outfiledir)
    outfilepath = os.path.join(outfiledir, name)
    #print(outfilepath)

    doc = SimpleDocTemplate(outfilepath, pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    Story=[]

    if school_name == 'DCP Harper Woods' or school_name =='DCP Redford':
      logo = Image(links[school_name], width=2*inch, height=0.42*inch, kind='proportional')
    else:
      logo = Image(links[school_name])

    
    NumberOfTotalAbsences = str(row.NumberOfTotalAbsences)
    story = []
    #Set the logo 
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=11>%s</font>' % formatted_time
     
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    ptext = '<font size=11>We are writing to you because <u>our records show that your child has missed more than 10 % of the school year to date.</u> During this challenging pandemic, we recognize the difficulties of transitioning to a very different school year in 2020. In light of the current COVID circumstances, class attendance and participation remains critical for a child’s educational growth and achievement, and we are committed to your child’s success.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=11>Our team will be following up this letter with a call to discuss reasons behind your student’s absence from remote learning. We recognize that the COVID pandemic has introduced a host of new challenges that our students and families have to navigat. Our goal is to work with you to identify the factors contributing to your child’s absenteeism and work together on a solution for improvement.  If absences continue, it will be necessary to schedule a meeting with school administration and commit to a further plan of action.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=11>Thank you for your partnership in addressing this matter, and thank you in advance for your dedication to ensuring that your child is set up for success in school this year. We are #BetterTogether, and we look forward to partnering with you to ensure that your child has a great year of learning and growth. </font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    ptext = '<font size=11>Sincerely,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=11>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=11>%s</font>'% (mails[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 


    #Second page barchart
    title_style = styles['Heading1']
    title_style.alignment = 1
    title = Paragraph("Absences Summary", title_style)
    Story.append(title)
    Story.append(Spacer(1, 24))
    pic = make_drawing(row)
    Story.append(pic)

    doc.build(Story)

In [ ]:
df[df.NumberOfTotalAbsences==df['NumberOfTotalAbsences'].max()]

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
144,DCP Harper Woods,50000891,1,Jones,Nevaeh,65.0,7.7


In [ ]:
df.groupby(['NameOfInstitution','GradeLevel'])[['StudentUniqueId']].count()

StudentUniqueId
NameOfInstitution     GradeLevel                 
Art in Motion         7                        22
                      8                       108
                      9                       127
CICS Bucktown         1                        24
                      2                        14
...                                           ...
River Heights Academy 5                        10
                      6                         7
                      7                         8
                      8                        10
                      K                        19

[70 rows x 1 columns]

# Running the code to generate the letter

In [ ]:
df.NameOfInstitution.unique()

array(['DCP Harper Woods', 'DCP Redford', 'CICS Prairie', 'CICS Bucktown',
       'CICS Irving Park', 'CICS West Belden',
       'Plymouth Educational Center', 'River Heights Academy',
       'Art in Motion'], dtype=object)

In [ ]:
#For English-Spanish letter, CICS schools
bi = df[(df.NameOfInstitution =='CICS Bucktown')|(df.NameOfInstitution == 'CICS Prairie')|(df.NameOfInstitution =='CICS Irving Park')|(df.NameOfInstitution =='CICS West Belden')]
dcp = df[(df.NameOfInstitution !='CICS Bucktown')|(df.NameOfInstitution != 'CICS Prairie')|(df.NameOfInstitution !='CICS Irving Park')|(df.NameOfInstitution !='CICS West Belden')]

In [ ]:
bi =bi.reset_index(drop=True)
dcp =dcp.reset_index(drop=True)

In [ ]:
dcp.tail()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
1632,Art in Motion,50199175,9,Williams,Jahzlynn,5.0,7.7
1633,Art in Motion,50123596,9,Williams,Tamiyah,46.0,7.7
1634,Art in Motion,50301647,9,Willis,Malaysia,44.0,7.7
1635,Art in Motion,50263275,9,Woods,Collin,25.0,7.7
1636,Art in Motion,45428796,9,Woods,Jordyn,40.0,7.7


In [ ]:
import time
start = time.clock() 
for i in range(bi.shape[0]):
  build_a_letter(bi.iloc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

Time spent in is:  14.267616


In [ ]:
#For English letter
start = time.clock() 
for i in range(dcp.shape[0]):
  build_english_letter(dcp.iloc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

In [ ]:
df.NameOfInstitution.unique()

array(['Art in Motion', 'Plymouth Educational Center'], dtype=object)

In [ ]:
#Sanity check
!ls /content/gdrive/My\ Drive/Chronic_absenteeism_letter/2020-12-16

'Art in Motion'     'CICS Prairie'	'DCP Redford'
'CICS Bucktown'     'CICS West Belden'	'Plymouth Educational Center'
'CICS Irving Park'  'DCP Harper Woods'	'River Heights Academy'


# Checking which way generate the letter faster

In [ ]:
df.head()

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
0,DCP Harper Woods,50000904,K,Collier,Megelle,39.0,7.0
1,DCP Harper Woods,50000911,K,Ware,Brian,42.0,7.0
2,DCP Harper Woods,50000789,K,Witherspoon,Ni'Lisa,42.0,7.0
3,DCP Harper Woods,50000272,5,Burton,Damijia,19.0,7.0
4,DCP Harper Woods,50000583,3,Campbell,Mackenzie,19.0,7.0


In [ ]:
mni = df[150:]

In [ ]:
dcp

,NameOfInstitution,StudentUniqueId,GradeLevel,LastSurname,FirstName,NumberOfTotalAbsences,AverageAbsence
0,DCP Harper Woods,50000904,K,Collier,Megelle,39.0,7.0
1,DCP Harper Woods,50000911,K,Ware,Brian,42.0,7.0
2,DCP Harper Woods,50000789,K,Witherspoon,Ni'Lisa,42.0,7.0
3,DCP Harper Woods,50000272,5,Burton,Damijia,19.0,7.0
4,DCP Harper Woods,50000583,3,Campbell,Mackenzie,19.0,7.0
...,...,...,...,...,...,...,...
321,DCP Redford,2018302,3,Verrett,Aden,12.0,7.0
322,DCP Redford,2018298,2,Franklin,Kyla,31.0,7.0
323,DCP Redford,2019027,K,Mccall,Arielle,31.0,7.0
324,DCP Redford,2019096,K,Scott,Chase,11.0,7.0


In [ ]:
#For English Letter only
import time
start = time.clock() 
for i in range(df.shape[0]):
  build_english_letter(df.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)

In [ ]:
import time
start = time.clock() 
for i in range(mni.shape[0]):
  build_a_letter(mni.loc[i], False)
elapsed = time.clock()
elapsed = elapsed - start
print("Time spent in is: ", elapsed)